In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Aug 10 09:27:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import os
import requests
import re

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
# download gpt2 models.  
gpt2.download_gpt2(model_name="124M")
gpt2.download_gpt2(model_name="355M")
#gpt2.download_gpt2(model_name="774M")
#gpt2.download_gpt2(model_name="1558M")

Fetching checkpoint: 1.05Mit [00:00, 575Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:01, 971kit/s]
Fetching hparams.json: 1.05Mit [00:00, 714Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:46, 10.6Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 662Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.14Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.16Mit/s]
Fetching checkpoint: 1.05Mit [00:00, 678Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:01, 950kit/s]
Fetching hparams.json: 1.05Mit [00:00, 604Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [03:27, 6.85Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 353Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:01, 951kit/s]
Fetching 

legacy:  
If dataset hasn't been created and splited to train & test dataset

In [ ]:
#  copy models to google drive if you want
!cp -r /content/models /content/drive/MyDrive 

In [ ]:
# load dataset
file_name = "HarryPotter.txt"

# if no dataset here, create a HarryPotter dataset
if not os.path.isfile(file_name):
	url = "https://raw.githubusercontent.com/YanWang20121865/MSc2021YanWang/main/Second%20update/Dataset/HarryPotter.txt"
	data = requests.get(url)
	
	with open(file_name, 'w') as f:
		f.write(data.text)

In [ ]:
!export PYTHONIOENCODING=UTF-8

In [ ]:
def train_test_split_fun(file_name, num):
  if 0 < num < 1:
    # open original txt file
    text = open(file_name+'.txt',"r", encoding='UTF-8').read()    
    # split each line of content in text file 
    b=re.split('\n',text)

    # check how many lines in total
    n=0
    for i in b:
      n+=1
    print(n)

    # generage train.txt and test.txt files
    f1 = open(file_name+'_train.txt','a')
    f2 = open(file_name+'_test.txt','a')
    m=0
    for i in b:
      m+=1
      if m<=n*num:    # train : test = num
        f1.write(i)
      else:
        f2.write(i)
    #close files
    f1.close()
    f2.close()
  else:
    print("The training data ratio must between 0 and 1.0! ")

In [ ]:
# code for spliting train & test dataset
num = 0.8
file_name = 'HarryPotter'
train_test_split_fun(file_name, num)

copy dataset from google drive here

In [4]:
# copy datasets 
!cp -r /content/drive/MyDrive/Dataset/* /content

For restart training:

In [5]:
# load checkpoint to google drive if in need( if you want to restart training from existed checkpoint)
!cp -r /content/drive/MyDrive/checkpoint/ /content/checkpoint/

In [6]:
# set file name and model_name for training and generating
file_name = 'dataset_top1000'
model_name = '355M'

###Train model:

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name+'_train.txt',
              model_name=model_name,
              steps=10000,
              #Set to fresh to start training from the base GPT-2, or set to latest to restart training from an existing checkpoint.
              restore_from='latest',   
              run_name='run_'+model_name+'_'+file_name,
              print_every=10,
              sample_every=250,
              save_every=500,
              
              )

In [9]:
# save checkpoint to google drive 
!cp -r /content/checkpoint/ /content/drive/MyDrive

###Generate story txt file:  

In [7]:
# load the model we trained
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run_'+model_name+'_'+file_name)

Loading checkpoint checkpoint/run_355M_dataset_top1000/model-69501
INFO:tensorflow:Restoring parameters from checkpoint/run_355M_dataset_top1000/model-69501


In [10]:
# generate txt files to save stories
# path for generated file
gen_file = '/content/drive/MyDrive/Generated Story/run_'+model_name+'_'+file_name+'/{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())
start = input("Starting sentence:")
gpt2.generate_to_file(sess,
                      run_name='run_'+model_name+'_'+file_name,
                      destination_path=gen_file,
                      prefix=start,  # input starting sentence
                      length=800,   # set max length of generated story
                      temperature=0.9,  # The higher the temperature, the crazier the text
                      nsamples=1,   # set number of generated stories, can generate several samples simultaneously
                      batch_size=1,
                      )


Starting sentence:Welcome to University of Limerick


In [ ]:
# may have to run twice to get file to download
files.download(gen_file)